In [1]:
import requests
import os 
import json
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
# Load apikey data

os.chdir('/Users/joemccann/MLProjects/alpaca_practice')
api_keys = json.load(open('apikey.json', 'r'))

In [20]:
# Historical data functions
def get_bar_data(api_keys, url, symbols, start='', end='', limit=1000, timeframe='1Day'):
    list_of_dfs = []
    
    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'timeframe':timeframe, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['bars'][symbol])
        df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        df['time'] = pd.to_datetime(df['time'])
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]
        df = df[['stock_name', 'time','open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]

        list_of_dfs.append(df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

def get_historical_quotes(api_keys, url, symbols, start='', end='', limit=1000):

    list_of_dfs = []

    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['quotes'][symbol])
        df.columns = ['ask_price', 'ask_size', 'ask_exchange', 'bid_price', 'bid_size', 'bid_exchange', 'condition_flag', 'timestamp', 'tape']
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]

        list_of_dfs.append(df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

# Latest data functions
def get_lastest_bar(api_keys, url, stocks):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }

    list_of_dfs = []

    for stock in stocks:
        params = {'symbols': stock}
        
        response = requests.get(url, headers=headers, params=params)
        
        data = response.json()
    
        last_bar_df = pd.DataFrame(data['bars']).T
        last_bar_df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        last_bar_df['stock_name'] = stock
        last_bar_df = last_bar_df[['stock_name'] + [col for col in last_bar_df.columns if col != 'stock_name']]
        list_of_dfs.append(last_bar_df)
    
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

In [21]:
# Get Bar Data

In [27]:
url = "https://data.alpaca.markets/v2/stocks/bars?"
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

get_bar_data(api_keys, url, symbols, start, end)

,stock_name,time,open,close,high,low,trade_count,volume,volume_weighted_avg
0,AMZN,2020-01-10 05:00:00+00:00,1905.37,1883.16,1906.94,1880.000,81924,3024534,1890.843009
1,AMZN,2020-01-13 05:00:00+00:00,1891.31,1891.30,1898.00,1880.800,78773,2992622,1890.957170
2,AMZN,2020-01-14 05:00:00+00:00,1885.88,1869.44,1887.11,1858.550,97323,3691756,1870.300572
3,AMZN,2020-01-15 05:00:00+00:00,1872.25,1862.02,1878.86,1855.090,80358,3056266,1864.995682
4,AMZN,2020-01-16 05:00:00+00:00,1882.99,1877.94,1885.59,1866.020,71727,2837080,1874.988524
...,...,...,...,...,...,...,...,...,...
2995,AAPL,2023-12-22 05:00:00+00:00,195.18,193.60,195.41,192.970,499840,37128763,194.101562
2996,AAPL,2023-12-26 05:00:00+00:00,193.61,193.05,193.89,192.830,488340,28921648,193.171311
2997,AAPL,2023-12-27 05:00:00+00:00,192.49,193.15,193.50,191.090,548205,48092035,192.567912
2998,AAPL,2023-12-28 05:00:00+00:00,194.14,193.58,194.66,193.170,472490,34056639,193.922138


In [ ]:
#Last Trade

In [25]:
last_url = "https://data.alpaca.markets/v2/stocks/bars/latest?"
stocks = ['AMZN', 'TSLA', 'AAPL']

get_lastest_bar(api_keys, last_url, stocks)

,stock_name,close,high,low,trade_count,open,time,volume,volume_weighted_avg
0,AMZN,157.8,157.8,157.62,194,157.62,2024-01-25T20:59:00Z,22949,157.711756
1,TSLA,183.45,183.45,183.45,1,183.45,2024-01-25T21:51:00Z,100,183.45
2,AAPL,194.19,194.2,193.92,252,194,2024-01-25T20:59:00Z,23714,194.048856


In [ ]:
# Historical Quotes

In [26]:
url = "https://data.alpaca.markets/v2/stocks/quotes?"
# symbols = 'TSLA'
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

get_historical_quotes(api_keys, url, symbols, start, end)

,stock_name,ask_price,ask_size,ask_exchange,bid_price,bid_size,bid_exchange,condition_flag,timestamp,tape
0,AMZN,1901.50,4,K,1901.00,1,T,[?],2020-01-10T00:00:49.393Z,C
1,AMZN,1901.50,4,K,1901.00,1,P,[?],2020-01-10T00:02:56.395Z,C
2,AMZN,1901.50,4,K,1901.00,1,T,[?],2020-01-10T00:03:19.479Z,C
3,AMZN,1901.50,3,K,1901.00,1,T,[?],2020-01-10T00:04:47.359Z,C
4,AMZN,1901.50,3,K,1901.04,1,K,[?],2020-01-10T00:04:50.045Z,C
...,...,...,...,...,...,...,...,...,...,...
2995,AAPL,311.99,2,P,311.94,45,P,[?],2020-01-10T12:00:44.404Z,C
2996,AAPL,311.99,1,P,311.94,45,P,[?],2020-01-10T12:00:46.144Z,C
2997,AAPL,311.99,1,P,311.94,43,P,[?],2020-01-10T12:00:47.305Z,C
2998,AAPL,311.99,1,P,311.94,33,P,[?],2020-01-10T12:00:47.817Z,C
